In [7]:
import time
import pandas as pd

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# chromedriver PATH
chromedriver_path = '../chromedriver'
service = Service(chromedriver_path)
chrome_options = Options()
chrome_options.add_argument("--disable-images")
driver = webdriver.Chrome(service=service, options=chrome_options)

In [9]:
driver.get("https://www.google.com/maps/place/Cervecer%C3%ADa+17punto10/@41.640786,-0.8981089,15z/data=!4m6!3m5!1s0xd5914d93b5647a5:0xf566713a4aa5e5b!8m2!3d41.6407845!4d-0.8922415!16s%2Fg%2F11cn3d_jtr?entry=ttu&g_ep=EgoyMDI0MDkxOC4xIKXMDSoASAFQAw%3D%3D")
wait = WebDriverWait(driver, 5)

try:
    accept_button = driver.find_element(By.XPATH, "//button[@aria-label='Aceptar todo']")
    accept_button.click()
    print("Se hizo clic en 'Aceptar todo'")
    time.sleep(1)  # Wait
except Exception as e:
    # If button doesnt appear, continue
    pass

In [11]:
# Extract starts resume
wait = WebDriverWait(driver, 10)
reviews_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')))

raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')

print(raw_html_reviews[0].text)

raw_html_stars_resumme = raw_html_reviews[0].find_elements(By.CSS_SELECTOR, 'tr.BHOKXe')

ratings = []
reviews_counts = []

for star in raw_html_stars_resumme:
    rating_text = star.get_attribute('aria-label')
    rating_parts = rating_text.split(',')
    stars = rating_parts[0].split()[0] 
    num_reviews = rating_parts[1].strip().split()[0].replace('.', '')
    ratings.append(int(stars))
    reviews_counts.append(int(num_reviews))

stars_resumme = pd.DataFrame({'stars': ratings, 'reviews': reviews_counts})
display(stars_resumme)

5
4
3
2
1
4,6
823 reseñas
  Las reseñas no se verifican

  
Escribir una reseña
  

Ordenar
Todas
panceta
85
postres
56
croquetas
36
carpaccio
23
raviolis
12
burrata
11
tarta de manzana
9
canelón
7
langostino
7
decoración
7


,stars,reviews
0,5,571
1,4,191
2,3,35
3,2,13
4,1,13


In [12]:
# Extract all visible reviews

reviewers = []
ratings = []
review_texts = []
review_dates = []
local_guides = []
text_backups = []

# Init a set for process reviews
processed_review_ids = set()

SCROLL_PAUSE_TIME = 120

last_height = driver.execute_script("return arguments[0].scrollHeight", reviews_container)
while True:
    # Extract all visible reviews
    raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
    
    # Extract data for each review
    for review in raw_html_reviews:
        try:
            review_id = review.get_attribute('data-review-id')
            
            # Verify if review has been processed
            if review_id in processed_review_ids:
                continue  # Omit processed review
            
            processed_review_ids.add(review_id)
            
            try:
                more_buttons = review.find_elements(By.CSS_SELECTOR, 'button.w8nwRe')
                for button in more_buttons:
                    if button.is_displayed():
                        button.click()
                        time.sleep(1)
            except Exception as e:
                print(f"No se encontró un botón 'Ver más' en esta reseña: {str(e)}")

            try: reviewer = review.find_element(By.CSS_SELECTOR, 'div.d4r55').text
            except: reviewer = ''

            try: review_text = review.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text
            except: review_text = ''

            try: rating = review.find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label')
            except: rating = ''

            try: review_date = review.find_element(By.CSS_SELECTOR, 'span.rsqaWe').text
            except: review_date = ''

            try: local_guide = review.find_element(By.CLASS_NAME, "RfnDt").text
            except: local_guide = ''

            print(reviewer + ' - ' + rating + ' - ' + review_date + ' - ' + review_text) 

            reviewers.append(reviewer)
            review_texts.append(review_text)
            ratings.append(rating)
            review_dates.append(review_date)
            local_guides.append(local_guide)

            text_backups.append(review.text)

        except Exception as e:
            print(f"Error al extraer una reseña: {str(e)}")

    # Scroll down the page
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", reviews_container)
    time.sleep(10)

    new_height = driver.execute_script("return arguments[0].scrollHeight", reviews_container)
    print(new_height)
    print(last_height)
    # If height doesn't change, wait to load more reviews.
    if new_height == last_height:
        print('No more reviews detected, forcing additional scroll')
        # Wait more reviews to be loaded
        time.sleep(SCROLL_PAUSE_TIME) 
        new_raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
        if len(new_raw_html_reviews) == len(raw_html_reviews): 
            print('No more reviews loaded, exiting scroll')
            break
    else:
        last_height = new_height

In [ ]:
collected_data = pd.DataFrame({
    'author': reviewers,
    'local_guide_info': local_guides,
    'rating': ratings,
    'review': review_texts,
    'date': review_dates,
    'text_backup': text_backups
})
display(collected_data)

In [ ]:
name = '17punto10.csv'

csv_file_path = '../data/raw/collected_reviews_'
collected_data.to_csv(csv_file_path + name, index=False)

stars_resumme_path = '../data/raw/resumme_'
collected_data.to_csv(stars_resumme_path + name, index=False)